<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/intel/Neural_speed_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install  mlflow pyngrok evaluate  bitsandbytes accelerate datasets transformers==4.41.2 tiktoken gguf transformers_stream_generator einops --quiet

#Pytorch Model from Hugginface

In [ ]:
from transformers import AutoTokenizer, TextStreamer, AutoModelForCausalLM

model_name = "Intel/neural-chat-7b-v3-1"     # Hugging Face model_id or local model
prompt = "Once upon a time, there existed a little girl,"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
inputs = tokenizer(prompt, return_tensors="pt").input_ids
streamer = TextStreamer(tokenizer)


model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)
model.generation_config.pad_token_id = tokenizer.pad_token_id

In [ ]:
import datetime
t1 = datetime.datetime.now()

outputs = model.generate(inputs, streamer=streamer, max_new_tokens=300, pad_token_id=tokenizer.eos_token_id)

t2 = datetime.datetime.now()


In [ ]:
(t2-t1).seconds, len(outputs[0])

In [ ]:
len(outputs[0])/ (t2-t1).seconds

# GGUF Model from Hugging Face


In [ ]:
import gc
import torch
# model.cpu()
# del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Specify the GGUF repo on the Hugginface
model_name = "TheBloke/Llama-2-7B-Chat-GGUF"
# Download the the specific gguf model file from the above repo
gguf_file = "llama-2-7b-chat.Q4_0.gguf"
# make sure you are granted to access this model on the Huggingface.
tokenizer_name = "meta-llama/Llama-2-7b-chat-hf"

prompt = "Once upon a time"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trust_remote_code=True)
inputs = tokenizer(prompt, return_tensors="pt").input_ids
streamer = TextStreamer(tokenizer)


model = AutoModelForCausalLM.from_pretrained(model_name, gguf_file = gguf_file, force_download=True)
model.generation_config.pad_token_id = tokenizer.pad_token_id

In [ ]:
t1 = datetime.datetime.now()

outputs = model.generate(inputs, streamer=streamer, max_new_tokens=300, pad_token_id=tokenizer.eos_token_id)

t2 = datetime.datetime.now()

print((t2-t1).seconds)

In [ ]:
(t2-t1).seconds, len(outputs[0])

In [ ]:
len(outputs[0])/ (t2-t1).seconds

# PyTorch Model qwen/Qwen-7B



In [ ]:

from transformers import AutoTokenizer, TextStreamer, AutoModelForCausalLM

model_name = "qwen/Qwen-7B"     #
prompt = "Once upon a time, there existed a little girl,"

model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True,  trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
inputs = tokenizer(prompt, return_tensors="pt").input_ids
model.generation_config.pad_token_id = tokenizer.pad_token_id

streamer = TextStreamer(tokenizer)

In [ ]:
import datetime
t1 = datetime.datetime.now()

outputs = model.generate(inputs, streamer=streamer, max_new_tokens=300, pad_token_id=tokenizer.eos_token_id)

t2 = datetime.datetime.now()

print((t2-t1).seconds)

In [ ]:
(t2-t1).seconds, len(outputs[0])

In [ ]:
len(outputs[0])/ (t2-t1).seconds